In [13]:
import os
from pathlib import Path
import pandas as pd
from IPython.display import display

pd.options.display.max_rows = 5
pd.options.display.max_columns = 1000


def read_data(path: Path) -> pd.DataFrame:
    return pd.read_json(path)


for i in Path(os.getenv("DATA_DIR")).rglob("*.json"):
    print(i)
    display(read_data(i))


/Users/yo/Developer/repo/sign-to-migrate/data/cpi.json


,countryiso3code,country_name,date,value
0,AFG,Afghanistan,2019,149.895975
1,AFG,Afghanistan,2018,146.522482
...,...,...,...,...
2523,ZWE,Zimbabwe,2011,103.466130
2524,ZWE,Zimbabwe,2010,100.000000


/Users/yo/Developer/repo/sign-to-migrate/data/unemployment_rate.json


,ref_area,source,indicator,sex,classif1,time,obs_value,obs_status,note_classif,note_indicator,note_source
0,Aruba,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Total,"Age (Youth, adults): 15+",2011,8.90,None,None,None,None
1,Aruba,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Male,"Age (Youth, adults): 15+",2011,8.60,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
43504,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Female,"Age (Youth, adults): 15-24",2004,8.13,None,None,None,Data reference period: June | Geographical cov...
43505,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Female,"Age (Youth, adults): 25+",2004,2.66,None,None,None,Data reference period: June | Geographical cov...


/Users/yo/Developer/repo/sign-to-migrate/data/women_in_senior_and_middle_position.json


...

,ref_area,source,indicator,sex,classif1,time,obs_value,obs_status,note_classif,note_indicator,note_source
0,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by dis...,Sex: Total,Disability status (Aggregate): Total,2021,5.679,None,None,None,Repository: ILO-STATISTICS - Micro data proces...
1,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by dis...,Sex: Total,Disability status (Aggregate): Persons with di...,2021,NaN,Unreliable,None,None,Repository: ILO-STATISTICS - Micro data proces...
...,...,...,...,...,...,...,...,...,...,...,...
7777,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by dis...,Sex: Female,Disability status (Aggregate): Persons with di...,2014,2.937,None,None,None,Repository: ILO-STATISTICS - Micro data proces...
7778,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by dis...,Sex: Female,Disability status (Aggregate): Persons without...,2014,5.627,None,None,None,Repository: ILO-STATISTICS - Micro data proces...


/Users/yo/Developer/repo/sign-to-migrate/data/informal_employment.json


,ref_area,source,indicator,sex,classif1,time,obs_value,obs_status,note_classif,note_indicator,note_source
0,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal e...,Sex: Total,"Economic activity (Agriculture, Non-Agricultur...",2021,86.395,None,None,None,Repository: ILO-STATISTICS - Micro data proces...
1,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal e...,Sex: Total,"Economic activity (Agriculture, Non-Agricultur...",2021,99.150,None,None,None,Repository: ILO-STATISTICS - Micro data proces...
...,...,...,...,...,...,...,...,...,...,...,...
57032,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal e...,Sex: Female,"Economic activity (Aggregate): Trade, Transpor...",2011,79.882,Break in series,None,Break in series: Methodology revised,Repository: ILO-STATISTICS - Micro data proces...
57033,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal e...,Sex: Female,Economic activity (Aggregate): Public Administ...,2011,63.919,Break in series,None,Break in series: Methodology revised,Repository: ILO-STATISTICS - Micro data proces...


/Users/yo/Developer/repo/sign-to-migrate/data/happiness.json


,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Dystopia Residual,Lower Confidence Interval,Upper Confidence Interval,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Health..Life.Expectancy.,Trust..Government.Corruption.,Dystopia.Residual
0,1.0,Finland,7.769,1.340,1.587,0.986,0.596,0.153000,0.393,2019,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Denmark,7.600,1.383,1.573,0.996,0.592,0.252000,0.410,2019,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,NaN,None,NaN,NaN,NaN,NaN,NaN,0.204435,NaN,2017,Burundi,None,NaN,NaN,NaN,NaN,0.629794,NaN,0.059901,NaN,NaN,NaN,NaN,154.0,2.905,3.074690,2.735310,0.091623,0.151611,0.084148,1.683024
781,NaN,None,NaN,NaN,NaN,NaN,NaN,0.280876,NaN,2017,Central African Republic,None,NaN,NaN,NaN,NaN,0.000000,NaN,0.270842,NaN,NaN,NaN,NaN,155.0,2.693,2.864884,2.521116,0.000000,0.018773,0.056565,2.066005
